In [1]:
import pandas as pd
import numpy as np

/home/mawdoo3/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/home/mawdoo3/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [16]:
data = pd.read_excel('Assignment_nearest neighbour.xlsx', index_col=0)
data.head()

,1648,5136,918,2824,3867,860,3712,2968,3525,4323,...,3556,5261,2492,5062,2486,4942,2267,4809,3853,2288
11: Star Wars: Episode IV - A New Hope (1977),NaN,4.5,5.0,4.5,4.0,4.0,NaN,5.0,4.0,5.0,...,4.0,NaN,4.5,4.0,3.5,NaN,NaN,NaN,NaN,NaN
12: Finding Nemo (2003),NaN,5.0,5.0,NaN,4.0,4.0,4.5,4.5,4.0,5.0,...,4.0,NaN,3.5,4.0,2.0,3.5,NaN,NaN,NaN,3.5
13: Forrest Gump (1994),NaN,5.0,4.5,5.0,4.5,4.5,NaN,5.0,4.5,5.0,...,4.0,5.0,3.5,4.5,4.5,4.0,3.5,4.5,3.5,3.5
14: American Beauty (1999),NaN,4.0,NaN,NaN,NaN,NaN,4.5,2.0,3.5,5.0,...,4.0,NaN,3.5,4.5,3.5,4.0,NaN,3.5,NaN,NaN
22: Pirates of the Caribbean: The Curse of the Black Pearl (2003),4.0,5.0,3.0,4.5,4.0,2.5,NaN,5.0,3.0,4.0,...,3.0,1.5,4.0,4.0,2.5,3.5,NaN,5.0,NaN,3.5


In [17]:
uu_sim = data.corr()

In [18]:
def get_top_users(uu_sim, target, n=5):
    target_corr = uu_sim.loc[target]
    top_neighbours = target_corr.nlargest(n+1).iloc[1:]
    return top_neighbours

In [19]:
def get_user_movie_score(movie, user):
    top_neighbours = get_top_users(uu_sim, user)
    ratings_sum = 0
    weight_sum = 0
    for user, w in zip(top_neighbours.index, top_neighbours.values):
        if np.isnan(movie[user]): 
            continue
        ratings_sum += movie[user] * w
        weight_sum += w
    if weight_sum == 0:
        return 0
    return ratings_sum/ weight_sum
        

In [20]:
pred_3712 = data.apply(get_user_movie_score,axis=1,args=(3712,))
pred_3712.sort_values(ascending=False)[:3]

641: Requiem for a Dream (2000)    5.000000
603: The Matrix (1999)             4.855924
105: Back to the Future (1985)     4.739173
dtype: float64

In [21]:
pred_3867 = data.apply(get_user_movie_score,axis=1,args=(3867,))
pred_89 = data.apply(get_user_movie_score,axis=1,args=(89,))

In [22]:
pred_3867.sort_values(ascending=False)[:3]

1891: Star Wars: Episode V - The Empire Strikes Back (1980)    4.760291
155: The Dark Knight (2008)                                    4.551454
122: The Lord of the Rings: The Return of the King (2003)      4.507637
dtype: float64

In [23]:
pred_89.sort_values(ascending=False)[:3]

238: The Godfather (1972)               4.894124
278: The Shawshank Redemption (1994)    4.882194
807: Seven (a.k.a. Se7en) (1995)        4.774093
dtype: float64

normalization

In [27]:
def get_user_movie_score_normalized(movie, user):
    top_neighbours = get_top_users(uu_sim, user)
    ratings_sum = 0
    weight_sum = 0
    user_rating_mean = data.loc[:,user].mean()
    for user, w in zip(top_neighbours.index, top_neighbours.values):
        if np.isnan(movie[user]): 
            continue
        movie_user_mean = data.loc[:,user].mean()
        ratings_sum += (movie[user]-movie_user_mean) * w
        weight_sum += w
    if weight_sum == 0:
        return 0
    return user_rating_mean + ratings_sum/ weight_sum

In [28]:
norm_pred_3867 = data.apply(get_user_movie_score_normalized,axis=1,args=(3867,))
norm_pred_89 = data.apply(get_user_movie_score_normalized,axis=1,args=(89,))

In [29]:
norm_pred_3867.sort_values(ascending=False)[:3]

1891: Star Wars: Episode V - The Empire Strikes Back (1980)    5.245509
155: The Dark Knight (2008)                                    4.856770
77: Memento (2000)                                             4.777803
dtype: float64

In [30]:
norm_pred_89.sort_values(ascending=False)[:3]

238: The Godfather (1972)               5.322015
278: The Shawshank Redemption (1994)    5.261424
275: Fargo (1996)                       5.241111
dtype: float64

In [31]:
norm_pred_3712 = data.apply(get_user_movie_score_normalized,axis=1,args=(3712,))
norm_pred_3712.sort_values(ascending=False)[:3]

641: Requiem for a Dream (2000)    5.900000
603: The Matrix (1999)             5.545567
105: Back to the Future (1985)     5.500585
dtype: float64